#Data Linkage

# Imports and Installs

This notebook accompanies the [article](https://pbpython.com/record-linking.html) on Practical Business Python

This notebook relies on [fuzzymatcher](https://github.com/RobinL/fuzzymatcher) and the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/about.html)


##Installs
installing fuzzymatcher and recordlinkage

###fuzzymatcher install

In [1]:
!pip install fuzzymatcher

     |████████████████████████████████| 50 kB 8.5 MB/s 
  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13919 sha256=96442e3ea018e6a15805400043c89fa1d315860341a021a1280972838419b2e0
  Stored in directory: /root/.cache/pip/wheels/1d/a8/cb/6f8902aa5457bd71344e00665c230e9c45255b3f57f2194a0f
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149863 sha256=c1087058fd72c89467861607288a01409f6c6e2fa9dd769a12d1abfef9100da8
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built metaphone python-Levenshtein


###recordlinkage install

In [2]:
!pip install recordlinkage

     |████████████████████████████████| 944 kB 31.1 MB/s 
     |████████████████████████████████| 137 kB 61.0 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.8.9-cp37-cp37m-linux_x86_64.whl size=73234 sha256=26e90478ffe478f241a73c5d39c2e5e2da8e5f041b8046e8f3a28c81712fcd05
  Stored in directory: /root/.cache/pip/wheels/32/a9/ef/5d8742e72deaf0d1de327a180d008c2c0299367581800ea73f
Successfully built jellyfish


##Imports

###Package Imports

In [3]:
import pandas as pd
from pathlib import Path
import fuzzymatcher
import recordlinkage

###Data File Imports and Sanitisation

loading and sanitising the data to use for fuzzy matching and record linkage

In [4]:
acm = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv',
    encoding='latin_1'
)
dblp = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv',
    encoding='latin_1'
)
perfect_matches = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/DBLP-ACM_perfectMapping.csv',
    encoding='latin_1'
)
perfect_matches = perfect_matches.sort_values(by=(['idACM', 'idDBLP']),
                                              ascending=False)
acm = acm.rename(columns={'title': 'titleACM', 'authors': 'authorsACM',
                          'id': 'idACM', 'year': 'yearACM', 'venue': 'venueACM'})
dblp = dblp.rename(columns={'title': 'titleDBLP', 'authors': 'authorsDBLP', 'id': 'idDBLP',
                            'year': 'yearDBLP', 'venue': 'venueDBLP'})
acm = acm.loc[:, ~acm.columns.str.contains('^Unnamed')]
acm['yearACM'] = acm['yearACM'].astype(str)
acm['authorsACM'] = acm['authorsACM'].astype(str)
acm = acm.reset_index(drop=True)
dblp = dblp.loc[:, ~dblp.columns.str.contains('^Unnamed')]
dblp['yearDBLP'] = dblp['yearDBLP'].astype(str)
dblp['authorsDBLP'] = dblp['authorsDBLP'].astype(str)
dblp = dblp.reset_index(drop=True)

#Fuzzy Matcher Approach

a simple fuzzy_left_join takes approximately 32s to run

##**fuzzy_matcher**
returns a set of linked id's which are believed to be matching using the fuzzy matcher library in the form of a dataframe

In [5]:
def fuzzy_matcher():
    left_on = ["authorsACM", "titleACM", "venueACM", "yearACM"]
    right_on = ["authorsDBLP", "titleDBLP", "venueDBLP", "yearDBLP"]
    matched_results = fuzzymatcher.fuzzy_left_join(acm,
                                                   dblp,
                                                   left_on,
                                                   right_on,
                                                   left_id_col='idACM',
                                                   right_id_col='idDBLP')
    cols = ["best_match_score", "idACM", "idDBLP"]
    matched_results = matched_results[cols].query('best_match_score >= 0.98').sort_values(by=['best_match_score'],
                                                                                          ascending=False)
    matched_results = matched_results.drop(['best_match_score'], axis=1)
    matched_results.sort_values(by=(['idACM', 'idDBLP']), ascending=False)
    matched_results = matched_results.reset_index(drop=True)
    matched_results.to_csv('fuzzy matcher results')
    return matched_results

###Obtaining the fuzzy matcher dataframe

In [6]:
fm_df = fuzzy_matcher()

To save as an excel spreadsheet.

In [7]:
#fm_df.sort_values(by=['idACM', 'idDBLP'],
#                        ascending=False).to_excel('merge_list.xlsx',
#                                                  index=False)

#Record Linkage Approach

Using block on the year as this is heavily unlikely to be different between the two compared datasets. Takes approximately 27s to run instead of the 6m 20s for the full approach. The full approach also didn't seem to lead to any better performance in terms of results from my testing so seems mostly irrelevant in this case.

##**record_linkage**

returns a set of linked id's which are believed to be matching using the record linkage library in the form of a dataframe

In [8]:
def record_linkage(threshold_scores):
  indexer = recordlinkage.Index()
  indexer.block(left_on="yearACM", right_on="yearDBLP")
  candidates = indexer.index(acm, dblp)
  compare = recordlinkage.Compare()
  compare.string('titleACM',
               'titleDBLP',
               threshold=threshold_scores[0],
               label='title')
  compare.string('venueACM',
               'venueDBLP',
               method='jarowinkler',
               threshold=threshold_scores[1],
               label='venue')
  compare.string('authorsACM',
               'authorsDBLP',
               method='jarowinkler',
               threshold=threshold_scores[2],
               label='authors')
  features = compare.compute(candidates, acm,
                           dblp)
  potential_matches = features[features.sum(axis=1) > 1].reset_index()
  potential_matches['Score'] = potential_matches.loc[:, 'title':'authors'].sum(axis=1)
  potential_matches = potential_matches.rename(columns={'level_0': 'idACM', 'level_1': 'idDBLP'})
  for index, row in potential_matches.iterrows():
    potential_matches.loc[index, 'idACM'] = acm.iloc[int(potential_matches.iloc[index]['idACM'])]['idACM']
    potential_matches.loc[index, 'idDBLP'] = dblp.iloc[int(potential_matches.iloc[index]['idDBLP'])]['idDBLP']
  potential_matches = potential_matches.drop(['title', 'venue', 'authors'], axis=1)
  return potential_matches

###Obtaining the record linkage dataframe

In [9]:
opt_scores = [0.6475, 0.9475, 0.5925]
rl_df = record_linkage(opt_scores)

To save as an excel spreadsheet.

In [10]:
#rl_df.sort_values(by=['idACM', 'idDBLP'],
#                        ascending=False).to_excel('merge_list.xlsx',
#                                                  index=False)

###Obtaining the realistic thresholds record linkage dataframe

here we are rounding to the closest 0.05 for a more realistic set of threshold scores for comparison to the optimised variant later

In [11]:
opt_scores = [0.65, 0.95, 0.6]
rl_r_df = record_linkage(opt_scores)

To save as an excel spreadsheet.

In [12]:
#rl_r_df.sort_values(by=['idACM', 'idDBLP'],
#                        ascending=False).to_excel('merge_list.xlsx',
#                                                  index=False)

##F1 score optimiser

Automation of threshold values for record linkage. i can be adjusted to run for extended periods adjusting opt_scores to find the highest f1 score possible eventually reaches an 'optimum' solution (to 0.0025 vals).

###**f1_score_optimise**

Note: takes approximately 40min (with i range of (0, 3)) to run
Returns adjusted threshold values which give a greater f1 score (or the same if no better result could be found)

In [19]:
def f1_score_optimise():
  opt_scores = [0.6475, 0.9475, 0.5925]
  rl_df = record_linkage(opt_scores)
  record_linkage_scores, record_linkage_results = score_finder(rl_df)
  for i in range(0, 3):
    print(opt_scores)
    for x in range(0, 3):
      new_scores = opt_scores
      new_scores[x] = new_scores[x] + 0.0025
      rl_df_new = record_linkage(new_scores)
      record_linkage_scores_new, record_linkage_results_new = score_finder(rl_df_new)
      if record_linkage_scores['f1'] > record_linkage_scores_new['f1']:
        new_scores[x] = new_scores[x] - 0.005
        rl_df_new = record_linkage(new_scores)
        record_linkage_scores_new, record_linkage_results_new = score_finder(rl_df_new)
        if record_linkage_scores['f1'] > record_linkage_scores_new['f1']:
          break
        else: 
          rl_df = rl_df_new
          record_linkage_scores = record_linkage_scores_new
          record_linkage_results = record_linkage_results_new
          opt_scores = new_scores 
      else:
        rl_df = rl_df_new
        record_linkage_scores = record_linkage_scores_new
        record_linkage_results = record_linkage_results_new
        opt_scores = new_scores 
  return rl_df, record_linkage_scores, record_linkage_results, opt_scores




In [22]:
# rl_o_df, rl_opimised_scores, rl_optimised_results, rl_optimal_values = f1_score_optimise()

###Commentary on this approach to score optimisation:
This approach doesn't use any data for validation or testing meaning that it is instead just optimising for the exact data and the results are unlikely to be the same for larger datasets of similar data. 

Therefore, I wouldn't recommend this approach in a real scenario but it does produce the "best" results in this case

As an additional note, this is also not even necessarily the best score possible as a local maximum for the score can easily be reached using this method rather than the true maximum since it only checks values on either side of itself rather than the whole range (as checking the whole range could take weeks which I, unfortunately, do not have or I would have loved to test this).  

#Score Finders

##**results_finder**

Finds the number of true positives, false positives and false negatives. takes 3m to run O(n<sup>2</sup>) time. Where n represents the matched results and the perfect data (more like O(n*m) but n's size ≈ m's size)

In [14]:
def results_finder(df):
    results = {}
    true_positives=0
    false_positives=0
    for index, row in df.iterrows():
        found = False
        for p_index, p_row in perfect_matches.iterrows():
          if row['idACM'] == p_row['idACM'] and row['idDBLP'] == p_row['idDBLP']:
            true_positives+=1 
            found = True
            break
        if found == False:
            false_positives+=1
    results['true_positives'] = true_positives
    results['false_positives'] = false_positives
    results['false_negatives'] = len(perfect_matches.index) - true_positives
    print("True Positives: " + str(results['true_positives']) 
    + " False Positives: " + str(results['false_positives']) 
    + " False Negatives: " + str(results['false_negatives']))
    return results

##**score_finder**

calculates the precision, recall and f1 scores using definitions from here: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9

In [15]:
def score_finder(df):
    results = results_finder(df)
    precision = results['true_positives'] / (results['true_positives'] + results['false_positives'])
    recall = results['true_positives'] / (results['true_positives'] + results['false_negatives'])
    f1 = 2 * ((precision * recall) / (precision + recall))
    print("Recall: " + str(recall) + " Precision: " +
          str(precision) + " F1: " + str(f1))
    return {"recall":recall, "precision":precision, "f1":f1}, results

#Scores

##Fuzzy Matcher Scores

takes approximately 3.5m to run


###Unadjusted fuzzy matcher scores

Takes approximately 3.5m to run

In [16]:
fuzzy_scores, fuzzy_results = score_finder(fm_df)

True Positives: 2144 False Positives: 10 False Negatives: 80
Recall: 0.9640287769784173 Precision: 0.9953574744661096 F1: 0.9794426678848789


###*Obseravtions on these results*

This approach produced an almost perfect precision score (0.995 3 s.f.) with only 10 false positives showing that it was very good at discarding false matches. Its recall (0.964 3s.f.) although comparatively unimpressive is still very high. Overall as the f1 score (0.979 3 s.f.) shows this method works exceedingly well on these datasets without much time needed for adjustment of what scores should be let through. 

##Record Linkage Scores



###Optimised recordlinkage scores

Takes approximately 3.5m to run

In [17]:
record_linkage_scores, record_linkage_results = score_finder(rl_df)

True Positives: 2071 False Positives: 46 False Negatives: 153
Recall: 0.9312050359712231 Precision: 0.9782711384034011 F1: 0.9541580281041234


###Rounded record linkage scores
Where the threshold values are rounded to the closest 0.05. Takes approximately 3.5m to run

In [18]:
record_linkage_realistic_scores, record_linkage_realistic_results = score_finder(rl_r_df)

True Positives: 2063 False Positives: 45 False Negatives: 161
Recall: 0.9276079136690647 Precision: 0.9786527514231499 F1: 0.9524469067405354


###*Observations on record linkage result differences*

This approach has pretty similar results when rounded or unrounded the rounded results being shown here mostly to present how a similar score is achievable with slightly less specific thresholds being used. Therefore, when talking about this approach in general I will refer to the optimised scores although the 0.2% difference in f1 score is still worth noting somewhat.

#Overall observations from this notebook

##*Observations on the differences between these 2 approaches*


###Record Linkage and Fuzzy Matcher score differences

The record linkage approach produced worse scores in both metrics than the fuzzy matcher approach. Precision still being excellent (0.978 3 d.p.) showing that the record linkage approach has potential. However, the main shortcoming is the recall (0.931 3 d.p.) which although not terrible is the 
lower score of the two almost doubling the number of false negatives from the fuzzy matcher approach (80 in FM and 153 in RL). To get to this stage also took around 24hrs of running the f1 score optimiser whilst the fuzzy matcher needed no adjustment. Overall, the record linkage approach managed to work less well in every way for this dataset except for a slightly faster run time (27s compared to 34s for the fuzzy matcher which is almost identical).

###Limitations

The amount of data present in this scenario is fairly limited with between 2000 and 3000 rows of data. This means that in this scenario creating a validation and testing data set would be hard as there isn't enough data to prove statistically significant when this data is split up in this way. This datasets constructions also leads to limitations as most of this data is relatively consistent between the sets with mild changes (such as DBLP.csv reporting a venue of 'SIGMOD record' whilst ACM.csv reports this as 'ACM SIGMOD Record' in many cases). 

###Possible improvements  

As stated in limitations making a testing and validation dataset would help to improve the ability to optimise the approaches and check if any changes to thresholds actually help or just make the model fit the data. Also finding if these datasets have any duplicates would help to improve the accuracy as at the moment there is no form of duplicate detection which could lead to errors and misleading results.